# DistilBERT Summary Comparison
Compare metrics from traditional and LLM runs.

In [ ]:
import os
import json
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

project_root = Path.cwd().resolve()
if project_root.name == 'notebooks':
    project_root = project_root.parent
os.chdir(project_root)
print('Project root:', project_root)

def load_summary(path: Path, label: str) -> pd.DataFrame:
    data = json.loads(path.read_text(encoding='utf-8'))
    df = pd.DataFrame(data)
    df['run_group'] = label
    return df

trad_df = load_summary(project_root / 'outputs/distilbert_imputed_runs/summary.json', 'traditional')
llm_df = load_summary(project_root / 'outputs/distilbert_imputed_llm_runs_third/summary.json', 'llm')
df = pd.concat([trad_df, llm_df], ignore_index=True, sort=False)
df['macro_f1'] = df['metrics'].apply(lambda m: m.get('macro_f1', float('nan')))
df['roc_auc'] = df['metrics'].apply(lambda m: m.get('roc_auc', float('nan')))
df['dp_diff'] = df['metrics'].apply(lambda m: m.get('dp_diff', float('nan')))
df[['dataset', 'run_group', 'macro_f1', 'roc_auc', 'dp_diff']].head()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='run_group', y='macro_f1')
sns.swarmplot(data=df, x='run_group', y='macro_f1', color='black', size=4)
plt.title('Macro F1 distribution by run group')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='run_group', y='roc_auc')
sns.swarmplot(data=df, x='run_group', y='roc_auc', color='black', size=4)
plt.title('ROC-AUC distribution by run group')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='run_group', y='dp_diff')
sns.swarmplot(data=df, x='run_group', y='dp_diff', color='black', size=4)
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.title('DP difference distribution by run group')
plt.tight_layout()
plt.show()

In [ ]:
summary = (df.groupby('run_group')[['macro_f1','roc_auc','dp_diff']]
           .agg(['mean','median','std']))
summary